# Cloud sandbox powered by FIWARE

## About the open source GPL3 license and copyright for this product

Copyright © 2024 Computate Limited Liability Company in Utah, USA

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

ADDITIONAL TERMS

As stated in section 7. c) and e) of the GPL3 license, 
"you may supplement the terms of this License with terms," 
Computate has added the following additional terms to the license: 

  7 c) Prohibiting misrepresentation of the origin of that material, and
    requiring that modified versions of such material be marked in
    reasonable ways as different from the original version;

  7 e) Declining to grant rights under trademark law for use of some
    trade names, trademarks, or service marks;

Please do not redistribute this course until you have built your own platform with these tools, 
separate from the computate.org platform, and reconfigure your fork of this repo to deploy 
your own platform instead of the computate.org platform. 

QUESTIONS

For questions about this open source license, please contact our public mailing list at computate@group.computate.org


## Switch to the Python 3.9 Kernel

When opening this Jupyter Notebook in VSCode, you will need to switch to the Jupyter 3.9 Kernel. 

# Install Keycloak with helm

In [ ]:
%%bash
oc create secret generic \
  --from-literal admin-password="$(tr -dc 'A-Za-z0-9' < /dev/urandom | head -c 16)" \
  keycloak-initial-admin
echo DONE

## Create Keycloak admin user

In [ ]:
%%bash
oc create secret generic \
  --from-literal password="$(tr -dc 'A-Za-z0-9' < /dev/urandom | head -c 16)" \
  postgres-pguser-keycloak
echo DONE

## Create Postgres password, user, and database for keycloak

In [ ]:
%%bash
oc exec pod/postgresql-0 -- env PGPASSWORD=$(oc get secret/postgresql -o jsonpath={.data.postgres-password} | base64 -d) psql -U postgres postgres -c \
  "create user keycloak password '$(oc get secret/postgres-pguser-keycloak -o jsonpath={.data.password} | base64 -d)'"
echo DONE

In [ ]:
%%bash
oc exec pod/postgresql-0 -- env PGPASSWORD=$(oc get secret/postgresql -o jsonpath={.data.postgres-password} | base64 -d) psql -U postgres postgres -c \
  "create database keycloak owner keycloak"
echo DONE

## Deploy Keycloak

For this next part, we will be deploying Keycloak with an external Route. 
You will need to check what is your OpenShift Developer Sandbox apps domain. 

Update the OPENSHIFT_APPS_DOMAIN environment variable below. 
It should look something like this: 

`apps.sandbox-m2.ll9k.p1.openshiftapps.com`. 

Run the commands below to install keycloak with helm. 

In [ ]:
%%bash
OPENSHIFT_APPS_DOMAIN=apps.sandbox-m2.ll9k.p1.openshiftapps.com

helm upgrade --install keycloak oci://registry-1.docker.io/bitnamicharts/keycloak --version 21.7.1 \
  --set postgresql.enabled=false \
  --set-json 'resources={"requests": {"cpu": "200m", "memory": "500Mi"}}' \
  --set externalDatabase.host=postgresql \
  --set externalDatabase.user=keycloak \
  --set externalDatabase.password=$(oc get secret/postgres-pguser-keycloak -o jsonpath={.data.password} | base64 -d) \
  --set externalDatabase.database=keycloak \
  --set externalDatabase.port=5432 \
  --set ingress.enabled=false \
  --set ingress.hostname=keycloak-$(oc project -q).$OPENSHIFT_APPS_DOMAIN \
  --set adminIngress.hostname=keycloak-$(oc project -q).$OPENSHIFT_APPS_DOMAIN \
  --set-json 'extraEnvVars=[{"name":"KC_HOSTNAME_URL","value":"https://keycloak-'"$(oc project -q)"'.'"$OPENSHIFT_APPS_DOMAIN"'"}, {"name":"KC_HOSTNAME_ADMIN_URL","value":"https://keycloak-'"$(oc project -q)"'.'"$OPENSHIFT_APPS_DOMAIN"'"}]' \
  --set ingress.tls=false \
  --set ingress.selfSigned=false \
  --set production=false \
  --set tls.enabled=false \
  --set tls.autoGenerated=false \
  --set image.tag=24.0.5 \
  --set auth.existingSecret=keycloak-initial-admin \
  --set auth.adminUser=admin \
  --set auth.passwordSecretKey=admin-password

oc create route edge --service keycloak --hostname "keycloak-$(oc project -q).$OPENSHIFT_APPS_DOMAIN"
echo DONE

### View Keycloak pod details
After deploying the Helm Chart, it will take a minute before the Keycloak pod is up and running. Run the command below until the Keycloak pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc get pod/keycloak-0
oc wait pod/keycloak-0 --for=condition=Ready --timeout=2m
oc get pod/keycloak-0
echo DONE

### View Keycloak pod logs
If your Keycloak pod does not reach the STATUS Running, you can run the command below to view the pod logs of Keycloak and check for other errors that may have occured. 

In [ ]:
%%bash
oc logs pod/keycloak-0
echo DONE

# Access the Keycloak dashboard

Access the Keycloak dashboard using the route. 

In [ ]:
%%bash
echo "https://$(oc get $(oc get route -l app.kubernetes.io/name=keycloak -o name) -o jsonpath={.spec.host})"
echo DONE

The username will be `admin` and run the command below to find the admin password.

In [ ]:
%%bash
oc get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d

## Next...
If you have successfully ran all of the commands above, congratulations, you are ready to move on to the next notebook in the course. 

- If you have additional questions or issues, please [create an issue for the course here](https://github.com/computate-org/computate/issues). 
- Otherwise, please continue to the next document [11-deploy-secrets.ipynb](11-deploy-secrets.ipynb). 
